In [1]:
import sqlite3
import pandas as pd

# Connect to an in-memory database
conn = sqlite3.connect(':memory:')

# For convenience 
cursor = conn.cursor()

In [2]:
# Creating a Practice Dataset in SQL

cursor.execute('''
CREATE TABLE netflix (
    show_id TEXT,
    title TEXT,
    type TEXT,
    release_year INTEGER,
    rating TEXT,
    country TEXT
)
''')

conn.commit()

In [3]:
sample_data = [
    ("s1", "Stranger Things", "TV Show", 2016, "TV-14", "United States"),
    ("s2", "White Chicks", "Movie", 2004, "PG-13", "United States"),
    ("s3", "Avatar: The Last Airbender", "TV Show", 2007, "TV-Y7", "United States"),
    ("s4", "Ne Zha", "Movie", 2019, "TV-14", "China"),
    ("s5", "Squid Game", "TV Show", 2021, "TV-MA", "South Korea")
]


cursor.executemany("INSERT INTO netflix VALUES (?, ?, ?, ?, ?, ?)", sample_data)
conn.commit()

In [4]:
pd.read_sql_query("SELECT * FROM netflix", conn)

,show_id,title,type,release_year,rating,country
0,s1,Stranger Things,TV Show,2016,TV-14,United States
1,s2,White Chicks,Movie,2004,PG-13,United States
2,s3,Avatar: The Last Airbender,TV Show,2007,TV-Y7,United States
3,s4,Ne Zha,Movie,2019,TV-14,China
4,s5,Squid Game,TV Show,2021,TV-MA,South Korea


In [5]:
# Practice queries:

# 1. Select all movie titles:

pd.read_sql_query(
    """
    SELECT * FROM netflix 
    WHERE type = 'Movie';
    """,
    conn
)

,show_id,title,type,release_year,rating,country
0,s2,White Chicks,Movie,2004,PG-13,United States
1,s4,Ne Zha,Movie,2019,TV-14,China


In [6]:
# 2. Shows released after 2018

pd.read_sql_query(
    """
    SELECT title, release_year
    FROM netflix
    WHERE release_year > 2017 AND type = 'TV Show';
    """,

    conn
)

,title,release_year
0,Squid Game,2021


In [7]:
# 3. Count how many shows per country

pd.read_sql_query(
    """
    SELECT country, COUNT(*) AS count 
    FROM netflix 
    GROUP BY country;
    """,
    conn
)
    

,country,count
0,China,1
1,South Korea,1
2,United States,3


In [8]:
# 4. Sort titles by newest -> oldest

pd.read_sql_query(
    """
    SELECT title, release_year
    FROM netflix
    ORDER BY release_year DESC;
    """,
    conn
)

,title,release_year
0,Squid Game,2021
1,Ne Zha,2019
2,Stranger Things,2016
3,Avatar: The Last Airbender,2007
4,White Chicks,2004


In [9]:
# 5. Most common rating 

pd.read_sql_query(
    """
    SELECT rating, COUNT(*) AS count
    FROM netflix 
    GROUP BY rating
    ORDER BY count DESC;
    """,
    conn
)

,rating,count
0,TV-14,2
1,TV-Y7,1
2,TV-MA,1
3,PG-13,1


In [11]:
cursor.execute('''
CREATE TABLE ratings_info (
    rating TEXT,
    description TEXT
)
''')

conn.commit()

In [12]:
ratings_data = [
    ("TV-14", "Parents strongly cautioned"),
    ("TV-MA", "Mature audiences only"),
    ("PG-13", "Parents cautioned for teens"),
    ("R", "Restricted - adults only")
]

cursor.executemany("INSERT INTO ratings_info VALUES (?, ?)", ratings_data)
conn.commit()

In [13]:
# To View the Ratings Table

pd.read_sql_query("SELECT * FROM ratings_info", conn)

,rating,description
0,TV-14,Parents strongly cautioned
1,TV-MA,Mature audiences only
2,PG-13,Parents cautioned for teens
3,R,Restricted - adults only


In [14]:
# INNER JOIN

pd.read_sql_query(
    """
    SELECT netflix.title, netflix.rating, ratings_info.description
    FROM netflix 
    INNER JOIN ratings_info
        ON netflix.rating = ratings_info.rating;
    """,
    conn
)

,title,rating,description
0,Stranger Things,TV-14,Parents strongly cautioned
1,White Chicks,PG-13,Parents cautioned for teens
2,Ne Zha,TV-14,Parents strongly cautioned
3,Squid Game,TV-MA,Mature audiences only


In [16]:
# Left JOIN - keeps everything from netflix, even if the rating is
# missing from ratings_info.

pd.read_sql_query(
    """
    SELECT netflix.title, netflix.rating, ratings_info.description
    FROM netflix
    LEFT JOIN ratings_info
        ON netflix.rating = ratings_info.rating;
    """,
    conn
)

,title,rating,description
0,Stranger Things,TV-14,Parents strongly cautioned
1,White Chicks,PG-13,Parents cautioned for teens
2,Avatar: The Last Airbender,TV-Y7,None
3,Ne Zha,TV-14,Parents strongly cautioned
4,Squid Game,TV-MA,Mature audiences only


In [17]:
# Analyst Style SQL Interview Questions Practice

In [19]:
# How many Movies vs TV Shows are in the table?

pd.read_sql_query(
    """
    SELECT type, COUNT(*) AS count
    FROM netflix
    GROUP BY type
    """,
    conn
)

,type,count
0,Movie,2
1,TV Show,3


In [20]:
# What are the top 3 countries by total number of titles?

pd.read_sql_query(
    """
    SELECT country, COUNT(*) AS count
    FROM netflix
    GROUP BY country
    ORDER BY count DESC
    LIMIT 3
    """,
    conn
)

,country,count
0,United States,3
1,South Korea,1
2,China,1


In [22]:
# What is the most common rating among Netflix titles?
pd.read_sql_query(
    """
    SELECT rating, COUNT(*) AS count
    FROM netflix
    GROUP  BY rating 
    ORDER BY count DESC
    LIMIT 1
    """,
    conn
)

,rating,count
0,TV-14,2


In [30]:
# List all titles along with their rating description
pd.read_sql_query(
    """
    SELECT netflix.title, netflix.rating, ratings_info.description
    FROM netflix
    INNER JOIN ratings_info 
        ON netflix.rating = ratings_info.rating
    """,
    conn
)

,title,rating,description
0,Stranger Things,TV-14,Parents strongly cautioned
1,White Chicks,PG-13,Parents cautioned for teens
2,Ne Zha,TV-14,Parents strongly cautioned
3,Squid Game,TV-MA,Mature audiences only


In [36]:
# Which year has the most Netflix titles released?

pd.read_sql_query(
    """
    SELECT release_year, COUNT(*) as count
    FROM netflix 
    GROUP BY release_year
    ORDER BY count DESC
    LIMIT 1;
    """,
    conn
)

,release_year,count
0,2021,1
